# Develop training and inference scripts for Script Mode

## Overview
In this notebook, we will learn how to develop training and inference scripts using HuggingFace framework. We will leverage SageMaker pre-build containers for HuggingFace (with PyTorch backend).

We chose to solve a typical NLP task - text classification. We will use `20 Newsgroups` dataset which assembles ~ 20,000 newsgroup documents across 20 different newsgroups (categories).

By the end of this notebook you will learn how to:
- prepare text corpus for training and inference using Amazon SageMaker;
- develop training script to run in pre-build HugginFace container;
- configure and schedule training job;
- develop inference code;
- configure and deploy real-time inference endpoint;
- test SageMaker endpoint.

Please note, that this notebook was tested on SageMaker Notebook instance with latest PyTorch dependencies installed (conda environment `conda_pytorch_latest_p36`). If you are using different environment, please make sure to install following Python dependencies via PIP or Conda installers:
- `scikit-learn`.
- `sagemaker`.
<br><br>


### Selecting Model Architecture
Our study task is to train model which can categorize newsgroup article based on its content into one of categories.

There are number of model architecture which can address this task. Existing State-of-the-art (SOTA) models are usually based on Transformer architecture. Autoregressive models like BERT and its various derivatives are suitable for this task. We will use concept known as `Transfer learning` where pre-trained model on one task is used for a new task with minimal modifications. 

As a baseline model we will use model architecture known as `DistilBERT` which provides high accuracy on wide variety of tasks and is considerably smaller than other models (for instance, original BERT model). To adapt model for classification task, we would need to add a classification layer which will be trained during our training to recognize articles.

![title](static/finetuning.png)

`HuggingFace Transformers` simplifies model selection and modification for fine-tuning:
- provides rich model zoo with number pre-trained models and tokenizers.
- has simple model API to modify baseline model for finetuning for specific task.
- implements inference pipelines, combining data preprocessing and actual inference together.

### Selecting SageMaker Training Containers

Amazon SageMaker supports HuggingFace Transformer framework for inference and trainining. Hence, we won't need to develop any custom container. Instead we will use `Script Mode` feature to provide our custom training and inference scripts and execute them in pre-build containers. In this example we will develop intution how to develop these scripts.

## Preparing Dataset
First of, we need to acquire `20 Newsgroups` dataset. For this, we can use `sklearn` module utility. To shorten training cycle, let's choose 6 newsgroup categories (original dataset contains 20). The datasets will be loaded into memory.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

# We select 6 out of 20 diverse newsgroups
categories = [
    "comp.windows.x",
    "rec.autos",
    "sci.electronics",
    "misc.forsale",
    "talk.politics.misc",
    "alt.atheism"
]

train_dataset = fetch_20newsgroups(subset='train',
                                  categories=categories,
                                  shuffle=True,
                                  random_state=42
                                 )
test_dataset = fetch_20newsgroups(subset='test',
                                  categories=categories,
                                  shuffle=True,
                                  random_state=42
                                 )

n=6 # arbitrary sample index
print(f"Number of training samples: {len(train_dataset['data'])}")
print(f"Number of test samples: {len(test_dataset['data'])}")

print(f"\n=========== Sample article for category {train_dataset['target'][n]} ============== \n")
print(f"{train_dataset['data'][n]}")


Now, we need to save selected datasets into files and upload resulting files to Amazon S3 storage.
SageMaker will download them to training container at training time.

In [17]:
import csv

for file in ['train_dataset.csv', 'test_dataset.csv']:
    with open(file, 'w') as f:
        w = csv.DictWriter(f, ['data', 'category_id'])
        w.writeheader()
        for i in range(len(train_dataset["data"])):
            w.writerow({"data":train_dataset["data"][i], "category_id":train_dataset["target"][i]})

`sagemaker.Session()` object provides a set of utilizities to manage interaction with Sagemaker and AWS services in general. Let's use it to upload our data files in dedicated S3 bucket.

In [ ]:
import sagemaker 

session = sagemaker.Session()
train_dataset_uri=session.upload_data("train_dataset.csv", key_prefix="newsgroups")
test_dataset_uri=session.upload_data("test_dataset.csv", key_prefix="newsgroups")

print(f"Datasets are available in following locations: {train_dataset_uri} and {test_dataset_uri}")

## Developing training script

When running SageMaker training job we need to provide training script. Additionally, we may provide any other dependencies. We can also install or modify Python packages installed in pre-built containers via `requirements.txt` file.

In this sample, we will use fairly new feature of HuggingFace framework to fine-tune multicategorical classifiers using Trainer API. Let's make sure that training container has newer HuggingFace Transformer library installed. For this, we create `requirements.txt` and specify minimal compatible version. We will provide this file to our SageMaker training job later.

In [11]:
!pygmentize 1_sources/requirements.txt

transformers >= 4.10


Next, we need to actually develop training script. See it's content prepared script `1_sources/trian.py` below. Here are several highlights of this script:
* At training time, SageMaker starts training by calling `user_training_script --arg1 value1 --arg2 value2 ...`. Here, arg1..N are hyperparameters provided by users as part of training job configuration. To correctly kick off training process in our script we need to include `a main guard` into our script (see line #100)
- To correctly capture hyperparameters, training script need to be able to parse command line arguments. We use Python argpars library to do it (see code snippet #104-#112)
* `train()` method is resposible for running end-to-end training job. It includes following components:
    - calling `_get_tokenized_dataset` to load and tokenize dataset using pretrained DistilBERT tokenizer from HuggingFace library;
    - loading and configuring DistilBERT model from HuggingFace model Zoo. Please note that we update default config for classification task to adjust for our chosen number of categories (line #80);
    - configure HuggingFace Trainer and start training process (lines #86-#93);
    - once training is done, we save trained model (line #97)
    


SageMaker Training toolkit setups up several environmental variables which comes handy when writing your training script:
- `"SM_CHANNEL_TRAIN"` and `"SM_CHANNEL_TEST"` are locations where data files are download before training begins;
- `"SM_OUTPUT_DIR"` is a directory for any output artifacts, SageMaker will upload this directory to S3 whether training job succeeds or failes;
- `"SM_MODEL_DIR"`is a directory to store resulting model artifacts, SageMaker will also upload the model to S3.

In [19]:
! pygmentize -O linenos=1 1_sources/train.py

0001: import argparse
0002: import os
0003: import json
0004: import torch
0005: import pandas as pd
0006: from transformers import (
0007:     DistilBertTokenizerFast,
0008:     DistilBertForSequenceClassification,
0009:     DistilBertConfig,
0010:     TrainingArguments,
0011:     Trainer,
0012: )
0013: 
0014: MODEL_NAME = "distilbert-base-uncased"
0015: NUM_LABELS = 6
0016: 
0017: 
0018: class CustomDataset(torch.utils.data.Dataset):
0019:     def __init__(self, encodings, labels):
0020:         self.encodings = encodings
0021:         self.labels = labels
0022: 
0023:     def __getitem__(self, idx):
0024:         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
0025:         item["label"] = torch.tensor(self.labels[idx])
0026:         return item
0027: 
0028:     def __len__(self):
0029:         return len(self.labels)
0030: 
0031: 
0032: def _get_tokenized_data():
0033:     """
0034:     Read data from CSV files and tokenize datasets for training
0035:   

## Running training job

Once we have training script and dependencies ready, we can proceed and schedule training job via SageMaker Python SDK.

We start with import of HuggingFace Estimator object and getting IAM execution role for our training job.

In [3]:
from sagemaker.huggingface.estimator import HuggingFace
from sagemaker import get_execution_role

role=get_execution_role()

Next, we need to define our hyperparameters of our model and training process. These variables will be passed to our script at training time.

In [17]:
hyperparameters = {
    "epochs":1,
    # 2 params below may need to updated if non-GPU instances is used for training
    "per-device-train-batch-size":16, 
    "per-device-eval-batch-size":64,
    "warmup-steps":100,
    "logging-steps":100,
    "weight-decay":0.01    
}

In [ ]:
estimator = HuggingFace(
    py_version="py36",
    entry_point="train.py",
    source_dir="1_sources",
    pytorch_version="1.7.1",
    transformers_version="4.6.1",
    hyperparameters=hyperparameters,
    instance_type="ml.p2.xlarge",
    instance_count=1,
    role=role
)


estimator.fit({
    "train":train_dataset_uri,
    "test":test_dataset_uri
})

## Developing Inference Code

Now that we have trained model, let's deploy it as SageMaker real-time endpoint. Similar to training job, we will use SageMaker pre-build HuggingFace container and will only provide our inference script. The inference requests will be handled by [Multi-Model Server](https://github.com/awslabs/multi-model-server) which exposes HTTP endpoint. 

When using pre-build inference containers, SageMaker automatically recognizes our inference script. According to SageMaker convention, inference script has to contain following methods:
- `model_fn(model_dir)` (lines #16-#45) is executed at container start time to load model in the memory. This method takes model directory as an input argument. You can use `model_fn()` to initiatilize other components of your inference pipeline, such as tokenizer in our case. Note, that HuggingFace Transformers has a convenient Pipeline API which allows to combine data pre-processing (in our case, text tokenization) and actual inference in a single object. Hence, instead of loaded model, we return inference pipeline (line #45).
- `transform_fn(inference_pipeline, data, content_type, accept_type)` is responsible for running actual inference (line #). Since we are communicating with end-client via HTTP, we also need to do payload deserialization and response serialization. In our sample example we expect JSON payload and return back JSON payload, however, this can be extended to any other formats based on the requirements (e.g. CSV, Protobuf).


Sometimes combining deserialization, inference, and serialization in a single method can be inconvenient. Alternatively, SageMaker supports more granular API:
- `input_fn(request_body, request_content_type)` runs deserialization.
- `predict_fn(deser_input, model)` performs predictions.
- `output_fn(prediction, response_content_type)` run serialization of predictions.

Note, that `transform_fn()` and `input_fn(); predict_fn(); output_fn()` are mutually exclusive.


In [1]:
! pygmentize -O linenos=1 1_sources/inference.py

0001: from transformers import (
0002:     pipeline,
0003:     DistilBertTokenizerFast,
0004:     DistilBertConfig,
0005:     DistilBertForSequenceClassification,
0006: )
0007: import torch
0008: import json
0009: 
0010: 
0011: MODEL_NAME = "distilbert-base-uncased"
0012: NUM_LABELS = 6 # number of categories
0013: MAX_LENGTH = 512 # max number of tokens model can handle
0014: 
0015: 
0016: def model_fn(model_dir):
0017:     """
0018:     Load required components (model, config and tokenizer) to constuct inference pipeline.
0019: 
0020:     This method is executed only once when SageMaker starts model server.
0021:     """
0022: 
0023:     # If CUDA device is present, then use it for inference
0024:     # otherwise fallback to CPU
0025:     device_id = 0 if torch.cuda.is_available() else -1
0026: 
0027:     tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)
0028:     config = DistilBertConfig()
0029:     config.num_labels = NUM_LABELS
0030: 
0031:     model = DistilBertFor

## Deploying Inference Endpoint

Now we are ready to deploy and test our Newsgroup Classification endpoint. We can use method `estimator.create_model()` to configure our model deployment parameters, specifically:
- define inference script and other dependencies which will be uploaded by SageMaker to endpoint;
- identify inference container. If you provide `transformers_version`, `pytorch_version` and `py_version` parameters, SageMaker will automatically find appropriate pre-built inference container (if it exists). Alternatively, you can provide `image_uri` to directly specify container image you wish to use.


In [4]:
from sagemaker.huggingface.estimator import HuggingFaceModel

model = estimator.create_model(role=role, 
                               entry_point="inference.py", 
                               source_dir="1_sources",
                               py_version="py36",
                               transformers_version="4.6.1",
                               pytorch_version="1.7.1"
                              )

Next, we define parameters of our endpoint such as number and type of instances behind it. Remember, SageMaker supports horizontal scaling of your inference endpoints! `model.deploy()` method starts inference deployment (which usually takes several minutes) and returns `Predictor` object to run inference requests.

In [5]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge"
)

-----------!

Now that endpoint is deployed, let's test it out! Note that we don't expect stellar performance, since model is likely undertrained because we only trained for single epoch to shorten training cycle. However, we expect that model will get most predictions right. 


In [ ]:
import random

for i in range(10):
    sample_id = random.randint(0, len(test_dataset['data']))
    prediction = predictor.predict([test_dataset['data'][sample_id]])
    print(f"Sample index: {sample_id}; predicted newsgroup: {prediction[0]['label']}; actual newsgroup: {test_dataset['target'][sample_id]}")



## Summary

In this notebook, you learned how to train and deploy custom HuggingFace model using **SageMaker Script mode**. Script mode provide a lot of flexibility for developers when it comes to development of training and inference scripts (as long as it's inline SageMaker conventions which we discussed in this notebook). You can also modify container runtime via `requirements.txt` if you need to install additional Python packages or upload custom code dependencies. 

However, there are scenarios when you need to have more control over your runtime environments. SageMaker allows you to extend pre-built containers or BYO containers. In next notebooks of this chapter we will learn when you need to consider modifying your containers and how to do it.